# Example for automated image genration using orthophtos and shape-files


## 1.) Input files:
- specify file paths
- choose webdriver (tested with chrome,firefox,safrie). (Default=chrome, download via: https://chromedriver.chromium.org/downloads, copy file to: copy to usr/local/bin/


In [1]:
from selenium import webdriver



script_dir = '/Users/flo/Proj/sat_hedge_detection/src/bio_generator'
shape_file_path = '../../data/Biotopkartierung/Biotopkartierung.shp'
wmts = "http://maps.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg"

## choose driver
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')
#driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')



## 2.) Load packages and functions



In [2]:
import os
import io
from PIL import Image

import geopandas as gpd

import folium
from folium.folium import Map
from folium.features import DivIcon

from pyproj import Transformer



def getText(temp_biotop):
    temp_nummer = temp_biotop['Nummer'][temp_biotop['Nummer'].keys()[0]]
    temp_gemnr = temp_biotop['gemnr'][temp_biotop['gemnr'].keys()[0]]
    temp_biotoptyp = temp_biotop['biotoptyp'][temp_biotop['biotoptyp'].keys()[0]]
    temp_bezeich = temp_biotop['bezeich'][temp_biotop['bezeich'].keys()[0]]
    text = "Nummer: " + str(temp_nummer) + "\n" + "GemNr: " + str(temp_gemnr) + "\n" + "Biotoptyp: " + str(
        temp_biotoptyp) + "\n" + "Bezeichnung: " + str(temp_bezeich)
    return text

def style_fcn(x):
    return {'stroke':True,'color': '#FF0000', 'fillColor': '#AARRGGBB','opacity':1, 'weight':2, 'line_cap':'round', 'fill':False}

def style_fnc_mask(x):
    return {'stroke':True,'color': '#FFFFFF', 'fillColor': '#FFFFFF','opacity':1, 'weight':10, 'line_cap':'round', 'fill':True, 'fillOpacity': 1}

def style_fcn_hull(x):
    return {'stroke':True,'color': '#008FFF', 'fillColor': '#AARRGGBB','opacity':1, 'weight':2, 'line_cap':'round', 'fill':False}


def style_fcn_env(x):
    return {'stroke':True,'color': '#F700FF', 'fillColor': '#AARRGGBB','opacity':1, 'weight':2, 'line_cap':'round', 'fill':False}



def biotop_center(temp_biotop):
    #get biotop pos
    center_temp = temp_biotop.geometry.centroid
    key = center_temp.keys()[0]
    return transformer.transform(center_temp[key].x,center_temp[key].y)

def biotop_current_map(temp_location, text, BIOTOP_BORDER, BIOTOP_MASK ,BIOTOP_DESCRIPTION, case):
    #TODO:  Linie weiter ausen umrunden lassen oder zusätzliche box
    #       Max Zoom tetsen (changed 18->20)

    BIOTOP_SCALE = BIOTOP_CONVEX = BIOTOP_ENV = 0

    m_temp = Map(tiles=None, location=[temp_location[1], temp_location[0]],
                 zoom_start=20,
                 prefer_canvas=True,
                 no_touch=True,
                 disable_3d=True,
                 attr="test",
                 control_scale=True,
                 zoom_control=False)

    atlas = folium.raster_layers.WmsTileLayer(url=wmts, layers="bmaporthofoto30cm",
                                              fmt='image/png',
                                              transparent=True,
                                              overlay=True).add_to(m_temp)

    if BIOTOP_BORDER:
        case = 'B_'+str(case)
        folium.GeoJson(temp_biotop, style_function=style_fcn).add_to(m_temp)
    # folium.GeoJson(temp_biotop_scaled,style_function=style_fcn).add_to(m_temp)
    # folium.GeoJson(temp_biotop_convex_hull, style_function=style_fcn_hull).add_to(m_temp)
    # folium.GeoJson(temp_biotop_envelope, style_function=style_fcn_env).add_to(m_temp)

    if BIOTOP_MASK:
        case = 'M_'+str(case)
        folium.GeoJson(temp_biotop, style_function=style_fnc_mask).add_to(m_temp)

    ## WRITE BIOTOP DESCIBTION IN IMAGE
    if BIOTOP_DESCRIPTION:
        folium.map.Marker([temp_location[1] + 0.001, temp_location[0] - 0.001],
                    icon=DivIcon(
                        icon_size=(150, 36),
                        icon_anchor=(0, 0),
                        html='<div style="font-size: 10pt;"><span style="color: #ff0000;">%s</span></div>' % text)).add_to(m_temp)
    if BIOTOP_SCALE:
        temp_biotop_scaled = temp_biotop.scale(xfact=2, yfact=2, zfact=0, origin='center')
    if BIOTOP_CONVEX:
        temp_biotop_convex_hull = temp_biotop.convex_hull
    if BIOTOP_ENV:
        temp_biotop_envelope = temp_biotop.envelope

    save_current_biotop2(m_temp, bio_i, case)
    #return m_temp

def save_current_biotop(m_temp, bio_number):
    png = m_temp._to_png()
    out = open(os.path.join(script_dir, '../../data/output_biotop/'+bio_number+'.png'), 'wb')
    bytes_written = out.write(png)
    out.close()

def create_bio_path(bio_i):
    temp_path = '../../data/output_biotop_dir/' + 'bio_' + bio_i + '/'
    if not os.path.exists(temp_path):
        os.makedirs(temp_path)
        return True
    return False

def save_current_biotop2(m_temp, bio_number,case):
    img_data = m_temp._to_png()
    img = Image.open(io.BytesIO(img_data))
    temp_path = '../../data/output_biotop_dir/' + 'bio_' + bio_number + '/'
    img.save(os.path.join(script_dir, temp_path + case + '_' + bio_number + '.png'))
    
    
print('Load: Biotopkartierung ', shape_file_path)
nReserve = gpd.read_file(os.path.join(script_dir, shape_file_path))

print('Load: wmts layer:' ,wmts)

print('Start: webdriver', driver)

Load: Biotopkartierung  ../../data/Biotopkartierung/Biotopkartierung.shp
Load: wmts layer: http://maps.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg
Start: webdriver <selenium.webdriver.chrome.webdriver.WebDriver (session="a864d6608648eea661822a5114d621f6")>


## 3.) Main script
- generates images (png) based on orthophtos and biotop shapefile 
     currently: biotoptyp=='8.1.1.1 + biotoptyp=='8.1.1.2
- create path: sat_hedge_detection/data/output_biotop_dir/

### Parameter to specify:
-  currently: todo



In [3]:
#script_dir = os.path.dirname(__file__)
#script_dir = '/Users/flo/Proj/sat_hedge_detection/src/bio_generator'



biotop_1 = nReserve.query("biotoptyp=='8.1.1.1'")
biotop_2 = nReserve.query("biotoptyp=='8.1.1.2'")


#driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')
#driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')


transformer = Transformer.from_crs(31258, 4326, always_xy=True)  # 1frpm 2to

len_bio_1 = biotop_1.shape[0]
len_bio_2 = biotop_2.shape[0]
len_totoal =  len_bio_1 + len_bio_2

biotop_key_nummer = biotop_1['Nummer']
biotop_keys = biotop_key_nummer.keys()
for i in range(0, len_bio_1-1):
    key_i = biotop_keys[i]
    bio_i = biotop_1['Nummer'][key_i]
    str_nummer = "\'" + bio_i + "\'"
    query_str = "Nummer==" + str(str_nummer)
    temp_biotop = biotop_1.query(query_str)
    temp_location = biotop_center(temp_biotop)

    PATH_NEW = create_bio_path(bio_i)
    if PATH_NEW:
        print("Process Biotop (1): ", i, "/", len_totoal, " ", bio_i)
        #m_temp = biotop_current_map(temp_location,text=getText(temp_biotop),BIOTOP_BORDER=False, BIOTOP_DESCRIPTION=False, case=1)
        biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=True,
                       case='1')
        biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=True, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=False,
                       case=1)
        biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = True, BIOTOP_DESCRIPTION=False,
                       case=1)
    #save_current_biotop2(m_temp, bio_i, 1)
    else:
        print("Already Existing Biotop (1): ", i, "/", len_totoal, " ", bio_i)


biotop_key_nummer = biotop_2['Nummer']
biotop_keys = biotop_key_nummer.keys()
for j in range(0, len_bio_2-1):
    key_i = biotop_keys[j]
    bio_i = biotop_2['Nummer'][key_i]
    str_nummer = "\'" + bio_i + "\'"
    query_str = "Nummer==" + str(str_nummer)
    temp_biotop = biotop_2.query(query_str)

    temp_location = biotop_center(temp_biotop)

    PATH_NEW = create_bio_path(bio_i)
    if PATH_NEW:
        print("Process Biotop (2): ", j + i, "/", len_totoal, " ", bio_i)
        biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=True,
                       case='2')
        biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=True, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=False,
                       case=2)
        biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = True, BIOTOP_DESCRIPTION=False,
                           case=2)

    #m_temp = biotop_current_map(temp_location,text=getText(temp_biotop),BIOTOP_BORDER=False, BIOTOP_DESCRIPTION=False)
    #save_current_biotop2(m_temp, bio_i, 2)
    else:
        print("Already Existing Biotop (2): ", i, "/", len_totoal, " ", bio_i)

Process Biotop (1):  0 / 646   564050102
Already Existing Biotop (1):  1 / 646   564050115
Already Existing Biotop (1):  2 / 646   565380128
Already Existing Biotop (1):  3 / 646   565380129
Already Existing Biotop (1):  4 / 646   564040266
Already Existing Biotop (1):  5 / 646   564040314
Already Existing Biotop (1):  6 / 646   564040321
Already Existing Biotop (1):  7 / 646   564040315
Already Existing Biotop (1):  8 / 646   564040317
Already Existing Biotop (1):  9 / 646   564040319
Already Existing Biotop (1):  10 / 646   564040293
Already Existing Biotop (1):  11 / 646   564040290
Already Existing Biotop (1):  12 / 646   564040289
Already Existing Biotop (1):  13 / 646   564040318
Already Existing Biotop (1):  14 / 646   564040281
Already Existing Biotop (1):  15 / 646   564040299
Already Existing Biotop (1):  16 / 646   564040301
Already Existing Biotop (1):  17 / 646   564040265
Already Existing Biotop (1):  18 / 646   564040163
Already Existing Biotop (1):  19 / 646   564040201

Already Existing Biotop (1):  170 / 646   565511917
Already Existing Biotop (1):  171 / 646   565275043
Already Existing Biotop (1):  172 / 646   565274982
Already Existing Biotop (1):  173 / 646   565511417
Already Existing Biotop (1):  174 / 646   565271202
Already Existing Biotop (1):  175 / 646   565271201
Already Existing Biotop (1):  176 / 646   565375042
Already Existing Biotop (1):  177 / 646   565375054
Already Existing Biotop (1):  178 / 646   565325033
Already Existing Biotop (1):  179 / 646   565325061
Already Existing Biotop (1):  180 / 646   565275029
Already Existing Biotop (1):  181 / 646   565325003
Already Existing Biotop (1):  182 / 646   565511403
Already Existing Biotop (1):  183 / 646   565324073
Already Existing Biotop (1):  184 / 646   565015965
Already Existing Biotop (1):  185 / 646   565015963
Already Existing Biotop (1):  186 / 646   565015062
Already Existing Biotop (1):  187 / 646   565505056
Already Existing Biotop (1):  188 / 646   565015977
Already Exis

Already Existing Biotop (2):  274 / 646   563010079
Already Existing Biotop (2):  274 / 646   563070059
Already Existing Biotop (2):  274 / 646   563070060
Already Existing Biotop (2):  274 / 646   563180022
Already Existing Biotop (2):  274 / 646   563010088
Already Existing Biotop (2):  274 / 646   565390210
Already Existing Biotop (2):  274 / 646   565350008
Already Existing Biotop (2):  274 / 646   565350043
Already Existing Biotop (2):  274 / 646   565350009
Already Existing Biotop (2):  274 / 646   565350010
Already Existing Biotop (2):  274 / 646   565350011
Already Existing Biotop (2):  274 / 646   565350086
Already Existing Biotop (2):  274 / 646   565350033
Already Existing Biotop (2):  274 / 646   565390270
Already Existing Biotop (2):  274 / 646   564010065
Already Existing Biotop (2):  274 / 646   564010057
Already Existing Biotop (2):  274 / 646   564010198
Already Existing Biotop (2):  274 / 646   564010089
Already Existing Biotop (2):  274 / 646   564010066
Already Exis

Already Existing Biotop (2):  274 / 646   565314961
Already Existing Biotop (2):  274 / 646   565241568
Already Existing Biotop (2):  274 / 646   565242941
Already Existing Biotop (2):  274 / 646   565491968
Already Existing Biotop (2):  274 / 646   565242942
Already Existing Biotop (2):  274 / 646   565131960
Already Existing Biotop (2):  274 / 646   565491565
Already Existing Biotop (2):  274 / 646   565491551
Already Existing Biotop (2):  274 / 646   565282995
Already Existing Biotop (2):  274 / 646   565283028
Already Existing Biotop (2):  274 / 646   565274975
Already Existing Biotop (2):  274 / 646   565275015
Already Existing Biotop (2):  274 / 646   565274993
Already Existing Biotop (2):  274 / 646   565274992
Already Existing Biotop (2):  274 / 646   565274991
Already Existing Biotop (2):  274 / 646   565274962
Already Existing Biotop (2):  274 / 646   565274967
Already Existing Biotop (2):  274 / 646   565011438
Already Existing Biotop (2):  274 / 646   565501498
Already Exis

KeyboardInterrupt: 